# Stokes 2D

Staggered-grid finite-difference implementation of the _Data-Assimilation Example from Glaciology_ given in [1].

In [ ]:
import numpy as np
from scipy import optimize, signal
import matplotlib.pyplot as plt

In [ ]:
# Parameters
alpha = 0.1/180*np.pi # inclination of the plane
height = 1.0e3        # height of the ice sheet (m)
length = 2.0e4        # length of the domain (m)
nx = 32               # number of grid points in x-direction
ny = 16               # number of grid points in y-direction
gamma = 1.0e4         # penalty parameter
rho = 910.0           # density of ice, kg m^-3
mu0 = 1.0             # initial viscosity
g = 9.81              # gravitational acceleration (m s^-2)
A_glen = 1.0e-16      # Glen flow parameter (Pa^-n a^-1)
n_glen = 3            # Glen exponent
N = 30                # number of coefficients in the trigonometric expansion of beta^2

In [ ]:
# Setup the grid
hx = length/(nx-1)
hy = height/(ny-1)
x = np.linspace(0,length,nx)
y = np.linspace(0,height,ny)

In [ ]:
# Initialize coefficients
p = np.zeros(N+1); p[0] = 1000.0
p_desired = np.zeros(N+1); p_desired[0] = 1000.0; p_desired[1] = 1000.0
print('p_desired =', p_desired)

In [ ]:
# Compute the friction coefficient beta^2
def beta_squared(p):
    res = p[0]
    for k in range(1, N//2+1):
        res += p[2*k-1]*np.sin(2*np.pi*k*x/length)
        res += p[2*k]*np.cos(2*np.pi*k*x/length)
    return res

plt.figure(1)
fig, ax = plt.subplots()
ax.plot(x, beta_squared(p),label="initial")
plt.plot(x, beta_squared(p_desired), label="desired")
plt.xlim([0, length])
legend = ax.legend()
plt.xlabel(r'$x$')
plt.ylabel(r'$\beta^2$')
plt.show()

In [ ]:
beta2 = beta_squared(p)
beta2

In [ ]:
# Numbering scheme
indices = np.arange(3*(nx-1)*(ny-1))
u_indices = 3*np.arange((nx-1)*(ny-1))
v_indices = u_indices+1
p_indices = u_indices+2

In [ ]:
# print(u_indices)
# print(v_indices)
# print(p_indices)

In [ ]:
# Define indices
node_indices = np.arange((nx-1)*(ny-1)).reshape((ny-1), (nx-1), order="F")
node_indices_top = node_indices[0,:]
node_indices_inner = node_indices[1:-1,:].flatten()
node_indices_bottom = node_indices[-1,:]

In [ ]:
# print(node_indices)
# print(node_indices_top)
# print(node_indices_inner)
# print(node_indices_bottom)

In [ ]:
# Define indexing functions
def node_index(i, j):
    return j*(ny-1)+i

def u_index(i, j):
    n = 3*(nx-1)*(ny-1)
    return (3*node_index(i, j))%n

def v_index(i, j):
    n = 3*(nx-1)*(ny-1)
    return (3*node_index(i, j) + 1)%n

def p_index(i, j):
    n = 3*(nx-1)*(ny-1)
    return (3*node_index(i, j) + 2)%n

In [ ]:
# Print info of matrix construction
def print_info(node, index):
    if verbose==True:
        if index%3==0:
            print("Adding x-Stokes equation with index {} to node {}".format(3*node, node))
        if index%3==1:
            print("Adding y-Stokes equation with index {} to node {}".format(3*node+1, node))
        if index%3==2:
            print("Adding continuity equation with index {} to node {}".format(3*node+2, node))

In [ ]:
# Construct the coefficient matrix
verbose = False
A = np.zeros((3*(nx-1)*(ny-1),3*(nx-1)*(ny-1)))
for j in range(nx-1):
    for i in range(ny-1):
        node = node_index(i, j)
        
        if node in node_indices_inner:
            # Add x-Stokes equations to inner nodes
            print_info(node, 3*node)
            A[u_index(i,j),u_index(i,j+1)] += 2/hx**2
            A[u_index(i,j),u_index(i,j)] -= 2/hx**2
            A[u_index(i,j),u_index(i,j)] -= 2/hx**2
            A[u_index(i,j),u_index(i,j-1)] += 2/hx**2
            A[u_index(i,j),u_index(i+1,j)] += 1/hy**2
            A[u_index(i,j),u_index(i,j)] -= 1/hy**2
            A[u_index(i,j),u_index(i,j)] -= 1/hy**2
            A[u_index(i,j),u_index(i-1,j)] += 1/hy**2
            A[u_index(i,j),p_index(i,j)] -= 1/hx
            A[u_index(i,j),p_index(i,j-1)] += 1/hx
            A[u_index(i,j),v_index(i+1,j)] += 1/(hx*hy)
            A[u_index(i,j),v_index(i+1,j-1)] -= 1/(hx*hy)
            A[u_index(i,j),v_index(i,j)] -= 1/(hx*hy)
            A[u_index(i,j),v_index(i,j-1)] += 1/(hx*hy)
            
            # Add y-Stokes equations to inner nodes
            print_info(node, 3*node+1)
            A[v_index(i,j),v_index(i+1,j)] += 2/hy**2
            A[v_index(i,j),v_index(i,j)] -= 2/hy**2
            A[v_index(i,j),v_index(i,j)] -= 2/hy**2
            A[v_index(i,j),v_index(i-1,j)] += 2/hy**2
            A[v_index(i,j),v_index(i,j+1)] += 1/hx**2
            A[v_index(i,j),v_index(i,j)] -= 1/hx**2
            A[v_index(i,j),v_index(i,j)] -= 1/hx**2
            A[v_index(i,j),v_index(i,j-1)] += 1/hx**2
            A[v_index(i,j),p_index(i,j)] -= 1/hy
            A[v_index(i,j),p_index(i-1,j)] += 1/hy
            A[v_index(i,j),u_index(i,j+1)] += 1/(hx*hy)
            A[v_index(i,j),u_index(i-1,j+1)] -= 1/(hx*hy)
            A[v_index(i,j),u_index(i,j)] -= 1/(hx*hy)
            A[v_index(i,j),u_index(i-1,j)] += 1/(hx*hy)
            
        if node in node_indices_top:
            # Add x-Stokes equations to top nodes
            print_info(node, 3*node)
            A[u_index(i,j),u_index(i,j+1)] += 2/hx**2
            A[u_index(i,j),u_index(i,j)] -= 2/hx**2
            A[u_index(i,j),u_index(i,j)] -= 2/hx**2
            A[u_index(i,j),u_index(i,j-1)] += 2/hx**2
            A[u_index(i,j),u_index(i+1,j)] += 1/hy**2
            A[u_index(i,j),u_index(i,j)] -= 1/hy**2
            A[u_index(i,j),u_index(i,j)] -= 1/hy**2
            A[u_index(i,j),u_index(i-1,j)] = 0 # ghost point
            A[u_index(i,j),p_index(i,j)] -= 1/hx
            A[u_index(i,j),p_index(i,j-1)] += 1/hx
            A[u_index(i,j),v_index(i+1,j)] += 1/(hx*hy)
            A[u_index(i,j),v_index(i+1,j-1)] -= 1/(hx*hy)
            A[u_index(i,j),v_index(i,j)] -= 1/(hx*hy) # TODO: top boundary point
            A[u_index(i,j),v_index(i,j-1)] += 1/(hx*hy) # TODO: top boundary point
            
            # Add y-Stokes equations to inner nodes
            print_info(node, 3*node+1)
            A[v_index(i,j),v_index(i+1,j)] += 2/hy**2
            A[v_index(i,j),v_index(i,j)] -= 2/hy**2
            A[v_index(i,j),v_index(i,j)] -= 2/hy**2
            A[v_index(i,j),v_index(i-1,j)] = 0 # TODO: top boundary point
            A[v_index(i,j),v_index(i,j+1)] += 1/hx**2
            A[v_index(i,j),v_index(i,j)] -= 1/hx**2
            A[v_index(i,j),v_index(i,j)] -= 1/hx**2
            A[v_index(i,j),v_index(i,j-1)] += 1/hx**2
            A[v_index(i,j),p_index(i,j)] -= 1/hy
            A[v_index(i,j),p_index(i-1,j)] = 0 # top boundary point
            A[v_index(i,j),u_index(i,j+1)] += 1/(hx*hy)
            A[v_index(i,j),u_index(i-1,j+1)] = 0 # TODO: top boundary point
            A[v_index(i,j),u_index(i,j)] -= 1/(hx*hy)
            A[v_index(i,j),u_index(i-1,j)] = 0 # TODO: top boundary point
        
        if node in node_indices_bottom:
            # Add x-Stokes equations to bottom nodes
            print_info(node, 3*node)
            A[u_index(i,j),u_index(i,j+1)] += 2/hx**2
            A[u_index(i,j),u_index(i,j)] -= 2/hx**2
            A[u_index(i,j),u_index(i,j)] -= 2/hx**2
            A[u_index(i,j),u_index(i,j-1)] += 2/hx**2
            A[u_index(i,j),u_index(i+1,j)] = 0 # ghost point
            A[u_index(i,j),u_index(i,j)] -= beta2[j]/hy # TODO: fix bottom boundary point
            A[u_index(i,j),u_index(i,j)] -= 1/hy**2
            A[u_index(i,j),u_index(i-1,j)] += 1/hy**2
            A[u_index(i,j),p_index(i,j)] -= 1/hx
            A[u_index(i,j),p_index(i,j-1)] += 1/hx
            A[u_index(i,j),v_index(i+1,j)] = 0 # bottom boundary point
            A[u_index(i,j),v_index(i+1,j-1)] = 0 # bottom boundary point
            A[u_index(i,j),v_index(i,j)] -= 1/(hx*hy)
            A[u_index(i,j),v_index(i,j-1)] += 1/(hx*hy)
            
            # Add y-Stokes equations to inner nodes
            print_info(node, 3*node+1)
            A[v_index(i,j),v_index(i+1,j)] = 0 # bottom boundary point
            A[v_index(i,j),v_index(i,j)] -= 2/hy**2
            A[v_index(i,j),v_index(i,j)] -= 2/hy**2
            A[v_index(i,j),v_index(i-1,j)] += 2/hy**2
            A[v_index(i,j),v_index(i,j+1)] += 1/hx**2
            A[v_index(i,j),v_index(i,j)] -= 1/hx**2
            A[v_index(i,j),v_index(i,j)] -= 1/hx**2
            A[v_index(i,j),v_index(i,j-1)] += 1/hx**2
            A[v_index(i,j),p_index(i,j)] -= 1/hy
            A[v_index(i,j),p_index(i-1,j)] += 1/hy
            A[v_index(i,j),u_index(i,j+1)] += 1/(hx*hy)
            A[v_index(i,j),u_index(i-1,j+1)] -= 1/(hx*hy)
            A[v_index(i,j),u_index(i,j)] -= 1/(hx*hy)
            A[v_index(i,j),u_index(i-1,j)] += 1/(hx*hy)
            
        if node in np.concatenate((node_indices_inner, node_indices_top), axis=0):
            # Add continuity equations to top and inner nodes
            print_info(node, 3*node+2)
            A[p_index(i,j),u_index(i,j+1)] = gamma/hx
            A[p_index(i,j),u_index(i,j)] = -gamma/hx
            A[p_index(i,j),v_index(i+1,j)] = gamma/hy
            A[p_index(i,j),v_index(i,j)] = -gamma/hy
            A[p_index(i,j),p_index(i,j)] = 0#-1
        
        if node in node_indices_bottom:
            # Add continuity equations to bottom nodes
            print_info(node, 3*node+2)
            A[p_index(i,j),u_index(i,j+1)] = gamma/hx
            A[p_index(i,j),u_index(i,j)] = -gamma/hx
            A[p_index(i,j),v_index(i+1,j)] = 0 # bottom boundary point
            A[p_index(i,j),v_index(i,j)] = -gamma/hy
            A[p_index(i,j),p_index(i,j)] = 0#-1

In [ ]:
# Visualize the matrix A
# plt.spy(A, markersize=1)
# np.savetxt("output/A.csv", A, delimiter=",")

In [ ]:
# Construct the right-hand side vector b
b = np.zeros(3*(nx-1)*(ny-1))
b[u_indices] = -rho*g*np.sin(alpha)
b[v_indices] = rho*g*np.cos(alpha)
b[3*node_indices_top+1] = 0

In [ ]:
# Solve the system Ax=b
solution = np.linalg.solve(A, b)

In [ ]:
# Extract values of u, v, and p from the solution
u_values = np.take(solution, u_indices)
v_values = np.take(solution, v_indices)
p_values = np.take(solution, p_indices)

In [ ]:
# Append first column to the last one
U_values = np.zeros((ny-1, nx))
U_values[:,:-1] = u_values.reshape((ny-1), (nx-1), order="F")
U_values[:,-1] = U_values[:,0]

# Append first column to the last one and a bottom row of zeros
V_values = np.zeros((ny, nx))
V_values[0:-1,:-1] = v_values.reshape((ny-1), (nx-1), order="F")
V_values[:,-1] = V_values[:,0]

# Compute the averages at the center of each cell
U_values = (U_values[:,1:]+U_values[:,:-1])/2
V_values = (V_values[1:,1:]+V_values[:-1,:-1])/2

# Obtain pressure at the center of each cell
P_values = p_values.reshape(ny-1, nx-1)

In [ ]:
# Plot results
fig, ax = plt.subplots()
c = ax.pcolor(x, y, U_values, cmap='RdBu')
cbar = fig.colorbar(c)
plt.xlabel("x")
plt.ylabel("y")

## References

[1] Granzow, G. (2014). A tutorial on adjoint methods and their use for data assimilation in glaciology. Journal of Glaciology, 60(221), 440-446. doi:10.3189/2014JoG13J205

[2] Becker, T. W. and B. J. P. Kaus (2018). Numerical Modeling of Earth Systems. An introduction to computational methods with focus on solid Earth applications of continuum mechanics. Lecture notes for USC GEOL557, v. 1.2.1, available online at http://www-udc.ig.utexas.edu/external/becker/Geodynamics557.pdf, accessed 01/2018.